$\DeclareMathOperator{\opdiv}{div}$

Hybridization Techniques
===

Discretizing an elliptic equation by a primal method leads to a positive definite matrix. Using a mixed method leads to a saddle point problem. This disadvantage can be overcome by the so called hybridization technique:

The idea is as follows: Break the normal-continuity of the RT - space, and reinforce it by another Lagrange parameter. This new Lagrange parameter is a polynomial on every mesh edge (or face, in 3D). 

The variational formulation is as follows:

Find: $\sigma_h \in RT_k^{dc}, u_h \in P^k, \widehat u_h \in P^k(\cup E)$ such that

$$
\begin{array}{ccccccl}
\int_\Omega \sigma_h \tau_h & + & \sum_T \int_T \opdiv \tau_h \, u_h & + & \sum_E \int_E [\tau_{h,n}] \widehat u_h & = & 0 & \forall \, \tau_h \\
\sum_T \int_T \opdiv \sigma_h \, v_h & & & & & = & \int f v_h & \forall \, v_h \\
\sum_E \int_E [\sigma_{h,n}] \widehat v_h & & & & & = & 0 & \forall \,  \widehat v_h
\end{array}
$$

This formulation gives the same solution as the mixed formulation, so we don't need an extra error analysis

The physical meaning of the Lagrange paramter $\widehat u_h$ is the primal variable, what can be seen by integration by parts of the first  equation.

Now, Dirichlet boundary conditions are set by constraining the $\widehat u$ on the Dirichlet boundary, and Neumann boundary conditions are formulated by $\int_{\Gamma_N} g \widehat v$. The hybridized formulation is thus similar to a primal method.

The discretization system has the form

$$
\left( \begin{array}{ccc}
A & B_1^T & B_2^T \\
B_1 & & \\
B_2 & & 
\end{array} \right)
\left( \begin{array}{c}
\sigma \\ u_1 \\ u_2 
\end{array} \right)
=
\left( \begin{array}{c}
0 \\ f \\ 0
\end{array} \right)
$$

The submatrix 

$$
\left( \begin{array}{cc}
A & B_1^T \\
B_1 & 0
\end{array} \right)
$$

is regular and block-diagonal, each block corresponds to an element. Thus it can be cheaply inverted.

If $u_2$ were known, we could compute the first to variables from $u_2$:

$$
\left( \begin{array}{c}
\sigma \\ u_1
\end{array} \right)
= 
\left( \begin{array}{cc}
A & B_1^T \\
B_1
\end{array} \right)^{-1} 
\left[
\left( \begin{array}{c} 0 \\ f \end{array} \right)
- \left( \begin{array}{c} B_2^T \\ 0 \end{array} \right) u_2 
\right]
$$

Plug this term into the third equation $B_2 \sigma = 0$ we obtain the system

$$
\left( B_2 \; \; 0 \right)
\left( \begin{array}{cc}
A & B_1^T \\
B_1
\end{array} \right)^{-1} 
\left( \begin{array}{c} B_2^T \\ 0 \end{array} \right)
\; \; u_2 = 
\left( B_2 \; \; 0 \right)
\left( \begin{array}{cc}
A & B_1^T \\
B_1
\end{array} \right)^{-1} 
\left( \begin{array}{c} 0 \\ f \end{array} \right)
$$


The matrix on the left hand side is symmetric positive definite. It behaves like a standard system matrix (e.g. for condition number), and  standard iterative methods and preconditioners can be used for solution.


The lowest order hybrid method has the same degrees of freedom as the non-conforming $P^1$ element. Here, the extension to higher order is straight forward.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

In [2]:
order=2
Sigma = HDiv(mesh, order=order, discontinuous=True)
V = L2(mesh, order=order-1)
Vhat = FacetFESpace(mesh, order=order, dirichlet="left|bottom")
# Sigma = HDiv(mesh, order=0, discontinuous=True)
# V = L2(mesh, order=0)
# Vhat = FacetFESpace(mesh, order=0, dirichlet="left|bottom")

X = Sigma*V*Vhat

sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

n = specialcf.normal(mesh.dim)

a = BilinearForm(X, eliminate_internal = True)
a += (-sigma*tau + div(sigma)*v + div(tau)*u)*dx
a += (sigma*n*vhat+tau*n*uhat) * dx(element_boundary=True)

c = Preconditioner(a, "bddc")
a.Assemble()

f = LinearForm(X)
f += v*dx
f.Assemble()

gfu = GridFunction(X)


f.vec.data += a.harmonic_extension_trans * f.vec

# gfu.vec.data = a.mat.Inverse(X.FreeDofs(True)) * f.vec
from ngsolve.solvers import CG
CG (mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, 
    printrates=True, maxsteps=200)

gfu.vec.data += a.harmonic_extension * gfu.vec
gfu.vec.data += a.inner_solve * f.vec

Draw (gfu.components[0], mesh, "sigma")
Draw (gfu.components[1], mesh, "u")
# Draw (gfu.components[2], mesh, "uhat")

CG iteration 1, residual = 0.3766721408060262     
CG iteration 2, residual = 0.10907102688107831     
CG iteration 3, residual = 0.02762373762050006     
CG iteration 4, residual = 0.009460904839439215     
CG iteration 5, residual = 0.0030401178721749075     
CG iteration 6, residual = 0.0008737003721822246     
CG iteration 7, residual = 0.00036488495834927087     
CG iteration 8, residual = 0.00010180163060399381     
CG iteration 9, residual = 4.454444795606642e-05     
CG iteration 10, residual = 1.0276081830023155e-05     
CG iteration 11, residual = 3.2066421202691073e-06     
CG iteration 12, residual = 7.934242177194006e-07     
CG iteration 13, residual = 2.7503719049183633e-07     
CG iteration 14, residual = 7.87016254760295e-08     
CG iteration 15, residual = 2.591403235842198e-08     
CG iteration 16, residual = 7.189732121846986e-09     
CG iteration 17, residual = 2.4887181875054425e-09     
CG iteration 18, residual = 7.169826660237978e-10     
CG iteration 19, resid

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204-53-g6741b8aa5', 'mesh_dim': 2, 'order2d':…

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204-53-g6741b8aa5', 'mesh_dim': 2, 'order2d':…

BaseWebGuiScene